# Recent Screenshots


In [9]:
import os
import requests
import time
import urllib.parse
import datetime
from IPython.core.display import display, HTML, clear_output

prefix = os.environ.get('API_PREFIX', 'http://192.168.45.91:90/intranet')

try:
    r = requests.get(prefix + "/api/stats/crawler/recent-activity", timeout=5 )
except:
    # Probably a time-out:
    pass
if r.status_code == 200:
    display(HTML('<div>'))
    html = '<div>'
    for ss in r.json()['screenshots']:
        params = {'url': ss[0], 'target_date': ss[1]}
        url = prefix + "/api/screenshots/?" + urllib.parse.urlencode(params)
        html += '<a href="%s" style="float: left;"><img src="%s" style="max-height:150px;"/></a>' % (url,url)
    html += '</div>'
    display(HTML(html))
else:
    display(HTML('ERROR at %s' % datetime.datetime.now()))



In [ ]:
%%html
<script>
setTimeout(() => {
  window.location.reload();
}, 30000); // refresh every 30s
</script>